In [1]:
import fitz

filename = '../data/samples/AMS13_A-E-100_Ground Floor RCP.pdf'

doc = fitz.open(filename)
page = doc[0]

In [2]:
## extract text from the page
text = page.get_text()
[print(item) for item in text.split('\n')]

1
2
3
D
C
C
E
E
B
B
A
A
4
F
F
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
7300
7300
7300
16700
7300
7300
9400
9400
7300
7300
7300
9400
9400
7300
7300
7300
9400
9400
7300
7300
7300
9400
9400
7300
7300
7300
9400
225400
5500
20800
4400
20800
5500
57000
G
H
J
K
M
P
R
O
165.5 m²
13.1B.02
ELECTRICAL ROOM
165.5 m²
13.1A.02
ELECTRICAL ROOM
677.5 m²
13.1B.01
COLO 1 CELL 2
672 m²
13.1A.01
COLO 1 CELL 1
196 m²
13.1B.03
MECHANICAL CORRIDOR
194.5 m²
13.2B.03
MECHANICAL CORRIDOR
194 m²
13.4B.03
MECHANICAL CORRIDOR
677.5 m²
13.2B.01
COLO 2 CELL 2
677.5 m²
13.2A.01
COLO 2 CELL 1
676 m²
13.3B.01
COLO 3 CELL 2
676 m²
13.3A.01
COLO 3 CELL 1
172.5 m²
13.2B.02
ELECTRICAL ROOM
172.5 m²
13.2A.02
ELECTRICAL ROOM
166 m²
13.3B.02
ELECTRICAL ROOM
166 m²
13.3A.02
ELECTRICAL ROOM
677.5 m²
13.4B.01
COLO 4 CELL 2
677.5 m²
13.4A.01
COLO 4 CELL 1
196 m²
13.1A.03
MECHANICAL CORRIDOR
194.5 m²
13.2A.03
MECHANICAL CORRIDOR
194 m²
13.3A.03
MECHANICAL CORRIDOR
194.5 m²
13.4A.03
MECHANICAL CORRIDOR


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [3]:
## scale ratio

scale_ratio_prefix = 'Scale 1 : '

for item in text.split('\n'):
    if scale_ratio_prefix in item:
        print(item)

Scale 1 : 250


In [11]:
paths = page.get_drawings()  # extract existing drawings
# this is a list of "paths", which can directly be drawn again using Shape
# -------------------------------------------------------------------------
#
# define some output page with the same dimensions
outpdf = fitz.open()
outpage = outpdf.new_page(width=page.rect.height, height=page.rect.width)
shape = outpage.new_shape()  # make a drawing canvas for the output page
# --------------------------------------
# loop through the paths and draw them
# --------------------------------------
for path in paths:
    # ------------------------------------
    # draw each entry of the 'items' list
    # ------------------------------------
    for item in path["items"]:  # these are the draw commands
        if item[0] == "l":  # line
            shape.draw_line(item[1], item[2])
        elif item[0] == "re":  # rectangle
            shape.draw_rect(item[1])
        elif item[0] == "qu":  # quad
            shape.draw_quad(item[1])
        elif item[0] == "c":  # curve
            shape.draw_bezier(item[1], item[2], item[3], item[4])
        else:
            raise ValueError("unhandled drawing", item)
    # ------------------------------------------------------
    # all items are drawn, now apply the common properties
    # to finish the path
    # ------------------------------------------------------
    shape.finish(
        fill=path["fill"],  # fill color
        color=path["color"],  # line color
        dashes=path["dashes"],  # line dashing
        even_odd=path.get("even_odd", True),  # control color of overlaps
        closePath=path["closePath"],  # whether to connect last and first point
        lineJoin=path["lineJoin"],  # how line joins should look like
        lineCap=max(path["lineCap"]),  # how line ends should look like
        width=path["width"],  # line width
        stroke_opacity=path.get("stroke_opacity", 1),  # same value for both
#         fill_opacity=path.get("fill_opacity", 1),  # opacity parameters
        )
# all paths processed - commit the shape to its page
shape.commit()
outpage.set_rotation(90)
outpdf.save("drawings-page-0.pdf")

TypeError: 'NoneType' object is not iterable